In [ ]:
# hw_location: where you want to run inference
#     "@cloud" to use DeGirum cloud
#     "@local" to run on local machine
#     IP address for AI server inference
# model_zoo_url: url/path for model zoo
#     cloud_zoo_url: valid for @cloud, @local, and ai server inference options
#     '': ai server serving models from local folder
#     path to json file: single model zoo in case of @local inference
# model_name: name of the model for running AI inference
# image_source: image source for inference
#     path to image file
#     URL of image
#     PIL image object
#     numpy array
hw_location = "@cloud"
model_zoo_url = "https://cs.degirum.com/degirum/timm_gender_model_test"
vehicle_det_model_name = "vehicle_detection--256x256_float_openvino_cpu_1"
vehicle_rec_model_name = "vehicle_recognition--72x72_float_openvino_cpu_1"
image_source = "cars.jpg"

In [ ]:
import degirum as dg, degirum_tools
from vehicle_det_postprocessor import VehicleDetPostprocessor
from vehicle_rec_preprocessor import VehicleRecPreprocessor
from vehicle_rec_postprocessor import VehicleRecPostprocessor

vehicle_det_zoo = dg.connect(hw_location, model_zoo_url, degirum_tools.get_token())
vehicle_rec_zoo = dg.connect(hw_location, model_zoo_url, degirum_tools.get_token())
# load vehicle detection AI model and vehicle recognition AI model
vehicle_det_model = vehicle_det_zoo.load_model(vehicle_det_model_name, input_image_format="RAW", custom_postprocessor = VehicleDetPostprocessor)
vehicle_rec_model = vehicle_rec_zoo.load_model(vehicle_rec_model_name, custom_postprocessor = VehicleRecPostprocessor)

In [ ]:
rec_H, rec_W = vehicle_rec_model.model_info.InputW[0],vehicle_rec_model.model_info.InputC[0]
preprocessor = VehicleRecPreprocessor(rec_H, rec_W)

In [ ]:
vehicle_det_res = vehicle_det_model(image_source)
vehicle_rec_preprocess = preprocessor.initialize(vehicle_det_res)

for image_crop, index in vehicle_rec_preprocess:
    result = vehicle_rec_model([image_crop])
    result.results[0]["score"] = vehicle_det_res.results[index]["score"]    
    vehicle_det_res.results[index]["label"] = result.results[0]["label"]

In [ ]:
vehicle_det_res.results